In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from statsmodels.distributions.empirical_distribution import ECDF
import adherer as ad

# Load the dataset
tidy = pd.read_csv('med-events.csv', sep='\t', header=0)

tidy.columns = ["pnr", "eksd", "perday", "ATC", "dur_original"]
tidy["eksd"] = pd.to_datetime(tidy["eksd"], format="%m/%d/%Y")
print(tidy)

def SEE(arg1):
    # Filter data for the given medicine
    C09CA01 = tidy[tidy["ATC"] == arg1].copy()
    
    # Sort by patient ID and prescription date
    C09CA01.sort_values(by=["pnr", "eksd"], inplace=True)
    
    # Compute previous prescription date
    C09CA01["prev_eksd"] = C09CA01.groupby("pnr")["eksd"].shift(1)
    
    # Drop rows where previous date is missing
    C09CA01.dropna(subset=["prev_eksd"], inplace=True)
    
    # Compute event interval (days between prescriptions)
    C09CA01["event_interval"] = (C09CA01["eksd"] - C09CA01["prev_eksd"]).dt.days
    
    # Select one random event per patient
    Drug_see_p1 = C09CA01.groupby("pnr").sample(n=1, random_state=1234)
    
    # ECDF computation
    ecdf = ECDF(Drug_see_p1["event_interval"])
    dfper = pd.DataFrame({"x": ecdf.x, "y": ecdf.y})
    
    # Retain first 80% of ECDF
    dfper = dfper[dfper["y"] <= 0.8]
    
    # Determine max event interval within the 80% threshold
    ni = dfper["x"].max()
    
    # Filter patients with event intervals within this range
    Drug_see_p2 = Drug_see_p1[Drug_see_p1["event_interval"] <= ni]
    
    # KDE estimation on log-transformed event intervals
    density = gaussian_kde(np.log(Drug_see_p2["event_interval"]))
    x_vals = np.linspace(min(np.log(Drug_see_p2["event_interval"])), max(np.log(Drug_see_p2["event_interval"])), 1000)
    y_vals = density(x_vals)

    # Standardize data for clustering
    a = np.column_stack((x_vals, y_vals))
    scaler = StandardScaler()
    a_scaled = scaler.fit_transform(a)
    
    # Find optimal clusters using silhouette method
    silhouette_scores = []
    cluster_range = range(2, 10)
    for k in cluster_range:
        kmeans = KMeans(n_clusters=k, random_state=1234, n_init=10)
        labels = kmeans.fit_predict(a_scaled)
        score = silhouette_score(a_scaled, labels)
        silhouette_scores.append(score)
    
    # Determine best cluster number
    max_cluster = cluster_range[np.argmax(silhouette_scores)]
    
    # Perform K-Means clustering
    kmeans = KMeans(n_clusters=max_cluster, random_state=1234, n_init=10)
    dfper["cluster"] = kmeans.fit_predict(dfper[["x"]])
    
    # Compute min, max, median for each cluster
    cluster_stats = dfper.groupby("cluster")["x"].agg(["min", "max", "median"]).reset_index()
    
    # Assign cluster based on event interval
    results = Drug_see_p1.merge(cluster_stats, how="cross")
    results["Final_cluster"] = np.where(
        (results["event_interval"] >= results["min"]) & (results["event_interval"] <= results["max"]),
        results["cluster"],
        np.nan
    )
    
    # Drop rows with no assigned cluster
    results.dropna(subset=["Final_cluster"], inplace=True)
    
    # Merge with original data
    Drug_see_p1 = Drug_see_p1.merge(results[["pnr", "median", "Final_cluster"]], on="pnr", how="left")
    
    # Fill missing values with most frequent cluster median
    most_common_cluster = results["Final_cluster"].mode()[0]
    most_common_median = results[results["Final_cluster"] == most_common_cluster]["median"].median()
    Drug_see_p1["median"].fillna(most_common_median, inplace=True)
    
    # Assign median duration to original dataset
    Drug_see_p0 = C09CA01.merge(Drug_see_p1[["pnr", "median"]], on="pnr", how="left")
    
    return Drug_see_p0


import seaborn as sns
import matplotlib.pyplot as plt

def see_assumption(data):
    data = data.sort_values(by=["pnr", "eksd"])
    
    # Compute previous prescription date and event interval
    data["prev_eksd"] = data.groupby("pnr")["eksd"].shift(1)
    data.dropna(subset=["prev_eksd"], inplace=True)
    data["Duration"] = (data["eksd"] - data["prev_eksd"]).dt.days
    
    # Count prescription number per patient
    data["p_number"] = data.groupby("pnr").cumcount() + 1
    
    # Boxplot of duration grouped by prescription number
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=data["p_number"], y=data["Duration"])
    plt.axhline(y=data["Duration"].median(), color="red", linestyle="dashed")
    plt.title("Prescription Count vs Duration")
    plt.xlabel("Prescription Number")
    plt.ylabel("Duration (Days)")
    plt.show()

medA = SEE("medA")
medB = SEE("medB")

see_assumption(medA)
see_assumption(medB)


      pnr       eksd  perday   ATC  dur_original
0       1 2033-04-26       4  medA            50
1       1 2033-07-04       4  medB            30
2       1 2033-08-03       4  medB            30
3       1 2033-08-17       4  medB            30
4       1 2033-10-13       4  medB            30
...   ...        ...     ...   ...           ...
1075  100 2034-03-05       6  medB            30
1076  100 2034-04-07       6  medB            30
1077  100 2034-04-26       6  medB            30
1078  100 2034-05-26       6  medB            30
1079  100 2034-07-09       4  medB            30

[1080 rows x 5 columns]


ValueError: Input X contains infinity or a value too large for dtype('float64').